In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv


In [ ]:
from collections import defaultdict

# from llama_index.embeddings.voyageai import VoyageEmbedding
import nest_asyncio
import optuna
import pandas as pd

# from pymilvus import MilvusClient
# from qdrant_client import QdrantClient, AsyncQdrantClient

from utils.retrieve import (
    objective,
    extract_question_ngrams,
)


In [ ]:
# configure
qa_filename = "question_list.csv"
ngram_size = 2  # use 2 instead of 3 so we don't skip 2-word header chunks
f_beta = 3  # weight recall 3 times as important as precision in f-score
n_trials = 25  # number of Optuna trials

pd.set_option("display.max_colwidth", None)


In [ ]:
# qa_df = pd.read_csv(f'data/{qa_filename}', na_filter=False)
qa_df = pd.read_csv(f"../data/qa_list/{qa_filename}", na_filter=False)
print(len(qa_df))
qa_df.head(3)


In [ ]:
# NOTE: we shouldn't include questions in the *test* set right now,
# but people are still adding the manual quotes,
# and since we have so few questions with manual quotes so far
# we will use all of them for this demo.

# keep only rows with at least 1 manual quote
qa_df = qa_df[qa_df["Quotes"].notna() & (qa_df["Quotes"] != "")]
print(len(qa_df))


In [ ]:
# generate bigrams (ngram size=2) for each manual quote
# and store them in the question_ngrams dictionary
question_ngrams = extract_question_ngrams(qa_df, ngram_size)


In [13]:
def get_objective_fn(documents, ngram_size, question_ngrams):
    def inner(trial):
        return objective(trial, documents, ngram_size, question_ngrams)

    return inner


In [15]:
# ask Optuna to find the best hyperparameters

study_name = "test"  # Unique identifier of the study.
storage_name = f"sqlite:///optuna-{study_name}.db"
print(
    f"To see a dashboard, open a terminal, activate the virtual environment, and run: optuna-dashboard {storage_name}"
)
study = optuna.create_study(
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
    direction="maximize",
)
study.optimize(
    get_objective_fn(
        documents=[], ngram_size=ngram_size, question_ngrams=question_ngrams
    ),
    n_trials=n_trials,
)

study.best_params


To see a dashboard, open a terminal, activate the virtual environment, and run: optuna-dashboard sqlite:///optuna-test.db


[I 2024-08-21 09:29:02,529] Using an existing study with name 'test' instead of creating a new one.
[W 2024-08-21 09:29:02,668] Trial 2 failed with parameters: {'embed_model': 'text-embedding-3-small', 'splitter': 'markdown', 'include_prev_next_rel': False, 'index': 'milvus', 'milvus_k': 74} because of the following error: KeyError('MILVUS_URI').
Traceback (most recent call last):
  File "/home/isaiaszc/pathway/pathway-indexer/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_14974/164494811.py", line 3, in inner
    return objective(trial, documents, ngram_size, question_ngrams)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/isaiaszc/pathway/pathway-indexer/utils/retrieve.py", line 257, in objective
    uri=os.environ["MILVUS_URI"], token=os.environ["MILVUS_TOKEN"]
        ~~~~~~~~~~^^^^^^^^^^^^^^
  File "<frozen os>", line 

KeyError: 'MILVUS_URI'